In [ ]:
# error prop with monte carlo

# Man sollte das einfach an satelliten ding anhängen können
# sofern man man beim gaussian fit auch die cov matrix als "gaussian_cov" ausliest

def i_noGauss_with_err(data_smooth, g_paras, g_paras_cov, time_axis):
    I, mu, sigma, A, B = g_paras
    err_I, err_mu, err_sigma, err_A, err_B = np.sqrt(np.abs(np.diag(g_paras_cov)))
    fitted_gaussian_linear = gaussian(time_axis, I, mu, sigma, A, B)
    fitted_gaussian = gaussian(time_axis, I, mu, sigma, 0, 0)
    i_no_linear = data_smooth - A*time_axis - B
    i_noGauss = i_no_linear/fitted_gaussian
    err_i_no_linear = np.sqrt(err_A**2 + err_B**2)
    dgaussdI = fitted_gaussian/I 
    dgaussdmu = (fitted_gaussian*((2*time_axis + 2*mu)/(2*sigma**2)))
    dgaussdsigma = fitted_gaussian*(time_axis -mu)**2/(4*sigma**3)
    #dgaussdsigma = fitted_gaussian*(np.dot(time_axis -mu, time_axis - mu))/(4*sigma**3)
    err_fitted_gaussian = np.sqrt( (dgaussdI* err_I)**2 + (dgaussdmu*err_mu)**2 + (dgaussdsigma*err_sigma)**2)
    #err_fitted_gaussian = np.sqrt( (dgaussdI* err_I)**2 + np.dot(dgaussdmu, dgaussdmu)*err_mu**2 + (dgaussdsigma*err_sigma)**2)

    err_i_noGauss = np.abs((np.sqrt((err_i_no_linear/i_no_linear)**2 + (err_fitted_gaussian**2/np.dot(fitted_gaussian, fitted_gaussian)))*i_noGauss))
    #err_i_noGauss = np.sqrt((err_i_no_linear**2/np.dot(intensity_no_linear, intensity_no_linear)) + (err_fitted_gaussian**2/np.dot(fitted_gaussian, fitted_gaussian)))*intensity_noGauss
    
    return i_noGauss, err_i_noGauss

samples = 1000
omega_samples = []

params_errs_g = np.sqrt(np.diag(gaussian_cov))

for _ in range(samples):
    sampled_g_params = np.random.normal(loc = gaussian_params, scale= params_errs_g)
    
    synth_data_noGauss, err_synth_data_noGauss = i_noGauss_with_err(data_smoothed, sampled_g_params, gaussian_cov, time_axis)

    rel_synth = synth_data_noGauss[rel_time_start:rel_time_stop]

    sampled_t_params, _ = curve_fit(
        theoretical_fit,
        time_axis_rel,
        intensity_values_rel_smooth,
        p0=initial_guess,
        sigma = np.diag(err_synth_data_noGauss[rel_time_start:rel_time_stop]),
        absolute_sigma = True,
        #bounds=(lower_bounds, upper_bounds),
        maxfev=50000
    )

    S_mc, omega_mc, phi_mc = sampled_t_params
    omega_samples.append(omega_mc)

omega_mean = np.mean(omega_samples)
omega_std = np.std(omega_samples)



In [ ]:
from uncertainties import ufloat
#literature vals + errs (Ref: https://ssd.jpl.nasa.gov/planets/phys_par.html and https://physics.nist.gov/cgi-bin/cuu/Value?bg)
R_lit = 6371.0084*10**3
err_R_lit = 0.0001*10**3
M_lit = 5.97217*10**24
err_M_lit = 0.00028*10**24
G_lit = 6.67430*10**(-11)
err_G_lit = 0.00015*10**(-11)

omega_lit = omega_mean
err_omega_lit = omega_std

R, G, M, omega, r = sp.symbols('R G M omega r')

eq = r**3 + R*r**2 - (G*M)/omega**2
sol = sp.solve(eq, r)

roots = sp.polys.polyroots.roots(sp.Poly(eq, r), r)

# Store results
numeric_solutions = []
numeric_errors = []

# Compute error propagation for each symbolic root
for sol in roots:
    if isinstance(sol, sp.Expr):  # Ensure it's symbolic
        # Compute partial derivatives
        dR = sp.diff(sol, R)
        dG = sp.diff(sol, G)
        dM = sp.diff(sol, M)
        domega = sp.diff(sol, omega)

        # Total error propagation formula
        err_r = sp.sqrt(
            (dR * err_R_lit)**2 +
            (dG * err_G_lit)**2 +
            (dM * err_M_lit)**2 +
            (domega * err_omega_lit)**2
        )

        # Evaluate numerical solutions
        numeric_solution = sol.subs({R: R_lit, G: G_lit, M: M_lit, omega: omega_lit}).evalf()
        numeric_error = err_r.subs({R: R_lit, G: G_lit, M: M_lit, omega: omega_lit}).evalf()

        numeric_solutions.append(numeric_solution)
        numeric_errors.append(numeric_error)

# Print results
for i, (sol, err) in enumerate(zip(numeric_solutions, numeric_errors)):
    print(f"Solution {i+1}: r = {sol} ± {err}")
